<table align="left">
    <tr>
        <td style="vertical-align: middle; padding-left: 0px; padding-right: 0px;">
            <a href="https://creativecommons.org/licenses/by/4.0/">
                <img src="https://licensebuttons.net/l/by/4.0/80x15.png" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 5px; padding-right: 0px;">
            <a href="https://opensource.org/licenses/MIT">
                <img src="https://img.shields.io/badge/License-MIT-green.svg" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 15px;">
            &copy; Guillaume Rongier
        </td>
    </tr>
</table>

# Using multiple components

This notebook starts from the basic example of the [first notebook](1_basic-example.ipynb) to show how to manage the stratigraphy when combining multiple StratigraPy components.

### Imports

Let's first import all the required packages and components:

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cmocean

from landlab.components import FlowDirectorMFD, FlowAccumulator

from stratigrapy import RasterModelGrid
from stratigrapy.components import SeaLevelCalculator, GravityDrivenDiffuser, WaterDrivenDiffuser

## 1. Adding gravity-driven diffusion

We'll use the same case as in the [first notebook](1_basic-example.ipynb), starting with the same simulation time:

In [ ]:
timestep = 100.
runtime = 500000.
n_iterations = int(runtime/timestep)

Then the grid:

In [ ]:
grid = RasterModelGrid((25, 30),
                       xy_spacing=(2500., 2500.),
                       number_of_classes=2,
                       initial_allocation=n_iterations//100 + 100,
                       number_of_layers_to_fuse=100,
                       number_of_top_layers=100,
                       fuse_continuously=True)

In [ ]:
grid.set_closed_boundaries_at_grid_edges(True, True, True, False)

The initial topography:

In [ ]:
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation += 0.003*(grid.y_of_node - 50000.)

The sources of water and sediments:

In [ ]:
idx = np.ravel_multi_index(((23, 23), (14, 15)), grid.shape)
water_influx = grid.add_zeros('water__unit_flux_in', at='node', clobber=True)
water_influx[idx] = 5000. # m/yr

In [ ]:
sediment_influx = grid.add_field('sediment__unit_flux_in',
                                 np.zeros((grid.number_of_nodes, 2)),
                                 clobber=True)
sediment_influx[idx] = [0.7*50000., 0.3*50000.] # m3/yr

The component controlling sea level:

In [ ]:
slc = SeaLevelCalculator(grid, wavelength=[100000., 10000.], amplitude=[25., 2.5])

The component controlling water flow:

In [ ]:
fd = FlowDirectorMFD(grid, partition_method='slope', diagonals=True)

In [ ]:
fa = FlowAccumulator(grid, flow_director=fd)

The component controlling sediment transport:

In [ ]:
wdd = WaterDrivenDiffuser(grid,
                          transportability_cont=[1e-8, 1e-8],
                          transportability_mar=[4e-10, 2e-10],
                          wave_base=15.,
                          max_erosion_rate_sed=1e-2,
                          max_erosion_rate_br=1e-12,
                          active_layer_rate=None,
                          bedrock_composition=[0.7, 0.3],
                          fields_to_track='bathymetric__depth')

Finally, we add a new component for gravity-driven diffusion, i.e., where sediments only moved based on the slope:

In [ ]:
gdd = GravityDrivenDiffuser(grid,
                            diffusivity_cont=[1e-2, 1e-2],
                            diffusivity_mar=[4e-3, 2e-3],
                            wave_base=15.,
                            max_erosion_rate=1e-2,
                            fields_to_track='bathymetric__depth')

We can now run the model with a slight change: we add the function to run the new diffusion component, with the option of updating the top layer instead of adding a new layer. That way we don't increase much the computational burden.

In [ ]:
for i in tqdm(range(n_iterations)):
    slc.run_one_step(timestep)
    fa.run_one_step()
    wdd.run_one_step(timestep)
    gdd.run_one_step(timestep, update_compatible=True)
    grid.stacked_layers.fuse(time=np.mean, bathymetric__depth=np.mean)
grid.stacked_layers.fuse(finalize=True, time=np.mean, bathymetric__depth=np.mean)

Let's visualize the result:

In [ ]:
fig, ax = plt.subplots()

raster_x = grid.x_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)

pc = ax.pcolormesh(raster_x, raster_y, raster_z, cmap=cmocean.cm.topo,
                   norm=mcolors.CenteredNorm(grid.at_grid['sea_level__elevation']))
fig.colorbar(pc, ax=ax, label='Elevation (m)')

ax.set(xlabel='y (m)', ylabel='y (m)', aspect='equal');

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3.5))

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
# Sea level
fill_sea = ax.fill_between(raster_y, raster_z, grid.at_grid['sea_level__elevation'],
                           color='#c6dbef', zorder=0)
# Bedrock
ax.fill_between(raster_y, raster_z, raster_z.min(), color='#d9d9d9')

# Sediments
pc = grid.plot_layers(ax, 'bathymetric__depth', cmap=cmocean.cm.deep)
fig.colorbar(pc[0], ax=ax, label='Water depth (m)')

ax.set(xlabel='y (m)', ylabel='z (m)');

## 2. Using only gravity-driven diffusion in the marine domain

The WaterDrivenDiffuser follows the same principle as in [Granjeon (1996)](https://theses.hal.science/tel-00648827v1): water keep flowing into the sea, although it's influence decreases with depth through a depth-dependent transportability. Other models, including [Landlab](https://landlab.csdms.io/tutorials/marine_sediment_transport/simple_submarine_diffuser_tutorial.html), use only a gravity-driven diffusion in the marine domain. Let's see how we can still mimic that behavior.

Grid, initial topography, sediment and water sources, sea level variations, and water flow remain the same:

In [ ]:
grid = RasterModelGrid((25, 30),
                       xy_spacing=(2500., 2500.),
                       number_of_classes=2,
                       initial_allocation=n_iterations//100 + 100,
                       number_of_layers_to_fuse=100,
                       number_of_top_layers=100,
                       fuse_continuously=True)

In [ ]:
grid.set_closed_boundaries_at_grid_edges(True, True, True, False)

In [ ]:
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation += 0.003*(grid.y_of_node - 50000.)

In [ ]:
idx = np.ravel_multi_index(((23, 23), (14, 15)), grid.shape)
water_influx = grid.add_zeros('water__unit_flux_in', at='node', clobber=True)
water_influx[idx] = 5000. # m/yr

In [ ]:
sediment_influx = grid.add_field('sediment__unit_flux_in',
                                 np.zeros((grid.number_of_nodes, 2)),
                                 clobber=True)
sediment_influx[idx] = [0.7*50000., 0.3*50000.] # m3/yr

In [ ]:
slc = SeaLevelCalculator(grid, wavelength=[100000., 10000.], amplitude=[25., 2.5])

In [ ]:
fd = FlowDirectorMFD(grid, partition_method='slope', diagonals=True)
fa = FlowAccumulator(grid, flow_director=fd)

Let's change the water-driven diffusion to have a low wave base, which means the transportability in the marine domain decreases quickly to have little effect. Note that we still keep a low value to ensure that sediments are indeed transported into the marine domain and don't just pile up along the shore:

In [ ]:
wdd = WaterDrivenDiffuser(grid,
                          transportability_cont=[1e-8, 1e-8],
                          transportability_mar=[4e-10, 2e-10],
                          wave_base=0.2,
                          max_erosion_rate_sed=1e-2,
                          max_erosion_rate_br=1e-12,
                          bedrock_composition=[0.7, 0.3],
                          fields_to_track='bathymetric__depth')

Then we use higher diffusivity in the marine domain for the gravity-driven diffusion to reproduce a similar architecture to what we had before:

In [ ]:
gdd = GravityDrivenDiffuser(grid,
                            diffusivity_cont=[1e-2, 1e-2],
                            diffusivity_mar=[1, 0.5],
                            wave_base=15.,
                            max_erosion_rate=1e-2,
                            fields_to_track='bathymetric__depth')

Let's run the simulation:

In [ ]:
for i in tqdm(range(n_iterations)):
    slc.run_one_step(timestep)
    fa.run_one_step()
    wdd.run_one_step(timestep)
    gdd.run_one_step(timestep, update_compatible=True)
    grid.stacked_layers.fuse(time=np.mean, bathymetric__depth=np.mean)
grid.stacked_layers.fuse(finalize=True, time=np.mean, bathymetric__depth=np.mean)

And visualize the result:

In [ ]:
fig, ax = plt.subplots()

raster_x = grid.x_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)

pc = ax.pcolormesh(raster_x, raster_y, raster_z, cmap=cmocean.cm.topo,
                   norm=mcolors.CenteredNorm(grid.at_grid['sea_level__elevation']))
fig.colorbar(pc, ax=ax, label='Elevation (m)')

ax.set(xlabel='y (m)', ylabel='y (m)', aspect='equal');

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3.5))

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 14]
# Sea level
fill_sea = ax.fill_between(raster_y, raster_z, grid.at_grid['sea_level__elevation'],
                           color='#c6dbef', zorder=0)
# Bedrock
ax.fill_between(raster_y, raster_z, raster_z.min(), color='#d9d9d9')

# Sediments
pc = grid.plot_layers(ax, 'bathymetric__depth', cmap=cmocean.cm.deep)
fig.colorbar(pc[0], ax=ax, label='Water depth (m)')

ax.set(xlabel='y (m)', ylabel='z (m)');